In [1]:
import sqlite3
import random

# Connect to the database
conn = sqlite3.connect("retail_dw.db")
cursor = conn.cursor()

# Generate synthetic customer data
names = ["Alice", "Bob", "Charlie", "Diana", "Ethan", "Fiona", "George", "Hannah"]
locations = ["Nairobi", "Mombasa", "Kisumu", "Eldoret"]
genders = ["Male", "Female"]

customers = []
for i in range(1, 11):  # 10 customers
    name = random.choice(names)
    location = random.choice(locations)
    age = random.randint(18, 65)
    gender = random.choice(genders)
    customers.append((i, name, location, age, gender))

# Insert into CustomerDim
cursor.executemany("""
    INSERT INTO CustomerDim (CustomerID, Name, Location, Age, Gender)
    VALUES (?, ?, ?, ?, ?);
""", customers)

conn.commit()
conn.close()

print("✅ CustomerDim populated.")


✅ CustomerDim populated.


In [2]:
conn = sqlite3.connect("retail_dw.db")
cursor = conn.cursor()

# Synthetic product data
products = [
    (1, "Laptop", "Electronics", 750.00),
    (2, "Smartphone", "Electronics", 500.00),
    (3, "Desk Chair", "Furniture", 120.00),
    (4, "Notebook", "Stationery", 5.00),
    (5, "Pen", "Stationery", 1.50),
    (6, "Water Bottle", "Accessories", 10.00)
]

cursor.executemany("""
    INSERT INTO ProductDim (ProductID, ProductName, Category, UnitPrice)
    VALUES (?, ?, ?, ?);
""", products)

conn.commit()
conn.close()

print("✅ ProductDim populated.")


✅ ProductDim populated.


In [3]:
from datetime import datetime, timedelta

conn = sqlite3.connect("retail_dw.db")
cursor = conn.cursor()

# Generate 30 days of time data
start_date = datetime(2025, 7, 1)
time_data = []

for i in range(30):
    date = start_date + timedelta(days=i)
    time_id = i + 1
    time_data.append((
        time_id,
        date.strftime("%Y-%m-%d"),
        date.month,
        (date.month - 1) // 3 + 1,  # Quarter
        date.year
    ))

cursor.executemany("""
    INSERT INTO TimeDim (TimeID, Date, Month, Quarter, Year)
    VALUES (?, ?, ?, ?, ?);
""", time_data)

conn.commit()
conn.close()

print("✅ TimeDim populated.")


✅ TimeDim populated.


In [4]:
def preview_table(table_name):
    conn = sqlite3.connect("retail_dw.db")
    cursor = conn.cursor()
    cursor.execute(f"SELECT * FROM {table_name} LIMIT 5;")
    rows = cursor.fetchall()
    print(f"\nPreview of {table_name}:")
    for row in rows:
        print(row)
    conn.close()

for table in ["CustomerDim", "ProductDim", "TimeDim", "SalesFact"]:
    preview_table(table)



Preview of CustomerDim:
(1, 'George', 'Kisumu', 53, 'Male')
(2, 'Diana', 'Eldoret', 20, 'Male')
(3, 'George', 'Nairobi', 39, 'Male')
(4, 'Hannah', 'Mombasa', 37, 'Female')
(5, 'Bob', 'Kisumu', 46, 'Female')

Preview of ProductDim:
(1, 'Laptop', 'Electronics', 750.0)
(2, 'Smartphone', 'Electronics', 500.0)
(3, 'Desk Chair', 'Furniture', 120.0)
(4, 'Notebook', 'Stationery', 5.0)
(5, 'Pen', 'Stationery', 1.5)

Preview of TimeDim:
(1, '2025-07-01', 7, 3, 2025)
(2, '2025-07-02', 7, 3, 2025)
(3, '2025-07-03', 7, 3, 2025)
(4, '2025-07-04', 7, 3, 2025)
(5, '2025-07-05', 7, 3, 2025)

Preview of SalesFact:
